<a href="https://colab.research.google.com/github/s-c-soma/deep_context_graph/blob/main/code/StatementChecker_ComputeSentenceSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Env Setup**


In [1]:


# Mount google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=False)


# create folder to write data to
#CIS545_FOLDER=os.path.join(DRIVE_MOUNT, 'My Drive', 'CIS545_2020')
#HOMEWORK_FOLDER=os.path.join(CIS545_FOLDER, 'Project')
#os.makedirs(HOMEWORK_FOLDER, exist_ok=True)

Mounted at /content/drive


In [2]:
!ls '/content/drive/Shared drives/1DeepContextGraph/1DeepContextGraph'

code  documents


In [3]:
file_path = '/content/drive/Shared drives/1DeepContextGraph/1DeepContextGraph/code/data/'

# Library

In [12]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 2.1MB 23.4MB/s 
     |████████████████████████████████| 1.2MB 61.4MB/s 
     |████████████████████████████████| 3.3MB 66.8MB/s 
     |████████████████████████████████| 901kB 61.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=51cb1a695c50d7abfc51d03c784f53a86d73f74dd297b73b7f837eca233063ef
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
Successfully built sentence-transformers


In [4]:
!pip install LexRank

In [11]:
#import lexrank.degree_centrality_scores

from lexrank import STOPWORDS, LexRank
from path import Path


In [7]:
import json
import os
# For caculating approximate time to process notebook (IGNORE)
import datetime
datetime.datetime.now()
import argparse
import pandas as pd
import numpy as np
import operator
import pickle as pkl 
import matplotlib.pyplot as plt
import nltk as nl
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import ParameterGrid
import statistics
import random
import warnings
from string import punctuation
from matplotlib import pyplot
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
import nltk
import re
import io
import requests
import time
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import nltk.sentiment
from sentence_transformers import SentenceTransformer, util
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


True

# Load Data

In [12]:
data_fox_claim = pd.read_csv(file_path+'/fox_claim_news_processed.csv',low_memory=False)
#data_all_news = pd.read_csv(file_path+'/statement_checker_all.csv',low_memory=False)
data_polifact = pd.read_csv(file_path+'/political_fact_checker_processed.csv',low_memory=False)


In [13]:
data_fox_claim.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'link', 'published', 'title', 'text',
       'site', 'text_distilled', 'topics', 'named_entity_recognition',
       'top_10_freq_word', 'pasts_of_speech'],
      dtype='object')

In [14]:
data_polifact.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'sources', 'urls', 'title', 'summaries',
       'dates', 'ratings', 'text', 'authors', 'publishdate', 'relatedlinks',
       'claims', 'text_distilled', 'topics', 'named_entity_recognition',
       'top_10_freq_word', 'pasts_of_speech'],
      dtype='object')

In [9]:
data_fox_claim['Unnamed: 0'][1]

1

In [15]:
data_fox_claim = data_fox_claim.rename({'Unnamed: 0': 'documentId'}, axis='columns')
data_polifact = data_polifact.rename({'Unnamed: 0': 'documentId'}, axis='columns')

# Methods

## Lemmatization
  
First, the raw words must be converted to root forms.  

In [16]:
def lemmatize(tokenized_words):
  text = [nltk.WordNetLemmatizer().lemmatize(word) for word in tokenized_words]
  return text


## Removing Stop words



In [17]:
english_stopwords = set(stopwords.words('english') + list(punctuation) + [''])

def remove_stopwords(tokenized_words):
  text = [word for word in tokenized_words if word not in english_stopwords]
  return text


## Custom Filtering

Some of the one or two-letter words from the tokenized words are also removed to further cleanse the raw text. 

In [18]:
whitelist = set(['ai', 'ax', 'ca', 'eu', 'go', 'io', 'la', 'ox', 'us', 'uk', 
                 'al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'fl', 'ga', 'hi', 
                 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma', 'mi',
                 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny',
                 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn',
                 'tx', 'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy' ])
def remove_too_short(tokenized_words):
  text = [word for word in tokenized_words if (len(word) >= 3 or word not in whitelist) ]
  return text

## LDA Analysis


In [19]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

def topics(tokenized_words):
    d = Dictionary([tokenized_words])
    c = [d.doc2bow(tokenized_words)]
    m = LdaModel(c, num_topics=1, id2word=d)
    return list(m.print_topics(num_words=2))
  


## Topics as Simple List of Words

A list of topic terms is compiled as show below. The coefficients in front of each word are dropped as part of simplification. The assumption is that the top two words comprising the topic, are both significant enough to be treated equally. It is important that the goal is to build a reliable prediction model. While there is a risk of oversimplification, if the final model results in a poor accuracy score, the coefficient can always be reintroduced here.

In [20]:
def parseTopics(topics):
   output = []
   words = topics[0][1].split( '+' )
   for word in words:
       output.append( word.split('*')[1].replace( '"', '' ) )
   return output

# Text Summarization-Transformer (not complete)

## SentenceTransformer Load

In [21]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Our input document we want to summarize
# As example, we take the first section from Wikipedia
document = """
New York City (NYC), often called simply New York, is the most populous city in the United States. With an estimated 2019 population of 8,336,817 distributed over about 302.6 square miles (784 km2), New York City is also the most densely populated major city in the United States. Located at the southern tip of the U.S. state of New York, the city is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass. With almost 20 million people in its metropolitan statistical area and approximately 23 million in its combined statistical area, it is one of the world's most populous megacities. New York City has been described as the cultural, financial, and media capital of the world, significantly influencing commerce, entertainment, research, technology, education, politics, tourism, art, fashion, and sports. Home to the headquarters of the United Nations, New York is an important center for international diplomacy.
Situated on one of the world's largest natural harbors, New York City is composed of five boroughs, each of which is a county of the State of New York. The five boroughs—Brooklyn, Queens, Manhattan, the Bronx, and Staten Island—were consolidated into a single city in 1898. The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York is home to more than 3.2 million residents born outside the United States, the largest foreign-born population of any city in the world as of 2016. As of 2019, the New York metropolitan area is estimated to produce a gross metropolitan product (GMP) of $2.0 trillion. If the New York metropolitan area were a sovereign state, it would have the eighth-largest economy in the world. New York is home to the highest number of billionaires of any city in the world.
New York City traces its origins to a trading post founded by colonists from the Dutch Republic in 1624 on Lower Manhattan; the post was named New Amsterdam in 1626. The city and its surroundings came under English control in 1664 and were renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city was regained by the Dutch in July 1673 and was subsequently renamed New Orange for one year and three months; the city has been continuously named New York since November 1674. New York City was the capital of the United States from 1785 until 1790, and has been the largest U.S. city since 1790. The Statue of Liberty greeted millions of immigrants as they came to the U.S. by ship in the late 19th and early 20th centuries, and is a symbol of the U.S. and its ideals of liberty and peace. In the 21st century, New York has emerged as a global node of creativity, entrepreneurship, and environmental sustainability, and as a symbol of freedom and cultural diversity. In 2019, New York was voted the greatest city in the world per a survey of over 30,000 people from 48 cities worldwide, citing its cultural diversity.
Many districts and landmarks in New York City are well known, including three of the world's ten most visited tourist attractions in 2013. A record 62.8 million tourists visited New York City in 2017. Times Square is the brightly illuminated hub of the Broadway Theater District, one of the world's busiest pedestrian intersections, and a major center of the world's entertainment industry. Many of the city's landmarks, skyscrapers, and parks are known around the world. Manhattan's real estate market is among the most expensive in the world. Providing continuous 24/7 service and contributing to the nickname The City that Never Sleeps, the New York City Subway is the largest single-operator rapid transit system worldwide, with 472 rail stations. The city has over 120 colleges and universities, including Columbia University, New York University, Rockefeller University, and the City University of New York system, which is the largest urban public university system in the United States. Anchored by Wall Street in the Financial District of Lower Manhattan, New York City has been called both the world's leading financial center and the most financially powerful city in the world, and is home to the world's two largest stock exchanges by total market capitalization, the New York Stock Exchange and NASDAQ.
"""

## Split the document into sentences

In [22]:
#Split the document into sentences
sentences = nltk.sent_tokenize(document)
print("Num sentences:", len(sentences))


Num sentences: 29


## Compute the sentence embeddings

In [34]:
#Compute the sentence embeddings
embeddings = model.encode(sentences) #, convert_to_tensor=True

## Compute the pair-wise cosine similarities

In [38]:
from scipy import spatial
#Compute the pair-wise cosine similarities
#cos_scores = util.pytorch_cos_sim(embeddings, embeddings).numpy()
cos_scores = 1- spatial.distance.cosine(embeddings[0], embeddings[0])

In [39]:




#Compute the centrality for each sentence
centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

#We argsort so that the first element is the sentence with the highest score
most_central_sentence_indices = np.argsort(-centrality_scores)


#Print the 5 sentences with the highest scores
print("\n\nSummary:")
for idx in most_central_sentence_indices[0:5]:
    print(sentences[idx].strip())

NameError: ignored

# Text summarization using NLTK: WordFrequency Algorithm

https://becominghuman.ai/text-summarization-in-5-steps-using-nltk-65b21e352b65

## Text-Input

In [46]:
document = """
New York City (NYC), often called simply New York, is the most populous city in the United States. 
With an estimated 2019 population of 8,336,817 distributed over about 302.6 square miles (784 km2), 
New York City is also the most densely populated major city in the United States. 
Located at the southern tip of the U.S. state of New York, the city is the center of the 
New York metropolitan area, the largest metropolitan area in the world by urban landmass. 
With almost 20 million people in its metropolitan statistical area and approximately 
23 million in its combined statistical area, it is one of the world's most populous megacities. 
New York City has been described as the cultural, financial, and media capital of the world, 
significantly influencing commerce, entertainment, research, technology, education, politics, 
tourism, art, fashion, and sports. Home to the headquarters of the United Nations, 
New York is an important center for international diplomacy.
Situated on one of the world's largest natural harbors, 
New York City is composed of five boroughs, each of which is a county of the State of New York. 
The five boroughs—Brooklyn, Queens, Manhattan, the Bronx, and Staten Island—were consolidated 
into a single city in 1898. The city and its metropolitan area constitute the premier gateway for
 legal immigration to the United States. As many as 800 languages are spoken in New York,
  making it the most linguistically diverse city in the world. New York is home to more than 3.2 million residents born outside the United States, the largest foreign-born population of any city in the world as of 2016. As of 2019, the New York metropolitan area is estimated to produce a gross metropolitan product (GMP) of $2.0 trillion. If the New York metropolitan area were a sovereign state, it would have the eighth-largest economy in the world. New York is home to the highest number of billionaires of any city in the world.
New York City traces its origins to a trading post founded by colonists from the 
Dutch Republic in 1624 on Lower Manhattan; the post was named New Amsterdam in 1626.
 The city and its surroundings came under English control in 1664 and were renamed New York 
 after King Charles II of England granted the lands to his brother, the Duke of York. The city was regained
  by the Dutch in July 1673 and was subsequently renamed New Orange for one year and three months; 
  the city has been continuously named New York since November 1674. New York City was the capital of the United States from 1785 until 1790, and has been the largest U.S. city since 1790. The Statue of Liberty greeted millions of immigrants as they came to the U.S. by ship in the late 19th and early 20th centuries, and is a symbol of the U.S. and its ideals of liberty and peace. In the 21st century, New York has emerged as a global node of creativity, entrepreneurship, and environmental sustainability, and as a symbol of freedom and cultural diversity. In 2019, New York was voted the greatest city in the world per a survey of over 30,000 people from 48 cities worldwide, citing its cultural diversity.
Many districts and landmarks in New York City are well known, including three of the world's
 ten most visited tourist attractions in 2013. A record 62.8 million tourists visited New York City in 2017. 
 Times Square is the brightly illuminated hub of the Broadway Theater District,
  one of the world's busiest pedestrian intersections, and a major center of the world's entertainment industry.
   Many of the city's landmarks, skyscrapers, and parks are known around the world. 
   Manhattan's real estate market is among the most expensive in the world. 
   Providing continuous 24/7 service and contributing to the nickname The City that Never Sleeps, 
   the New York City Subway is the largest single-operator rapid transit system worldwide, 
   with 472 rail stations. The city has over 120 colleges and universities, 
   including Columbia University, New York University, Rockefeller University, 
   and the City University of New York system, which is the largest urban public 
   university system in the United States. Anchored by Wall Street in the Financial District of Lower Manhattan, New York City has been called both the world's leading financial center and the most financially 
powerful city in the world, and is home to the world's 
two largest stock exchanges by total market capitalization, the New York Stock Exchange and NASDAQ.
"""


 
New York City (NYC), often called simply New York, is the most populous city in the United States. New York City has been described as the cultural, financial, and media capital of the world, 
significantly influencing commerce, entertainment, research, technology, education, politics, 
tourism, art, fashion, and sports. Home to the headquarters of the United Nations, 
New York is an important center for international diplomacy. As many as 800 languages are spoken in New York,
  making it the most linguistically diverse city in the world. New York is home to more than 3.2 million residents born outside the United States, the largest foreign-born population of any city in the world as of 2016. If the New York metropolitan area were a sovereign state, it would have the eighth-largest economy in the world. New York is home to the highest number of billionaires of any city in the world. New York City traces its origins to a trading post founded by colonists from the 
Dutch Republic in 1624 on Lower Manhattan; the post was named New Amsterdam in 1626. New York City was the capital of the United States from 1785 until 1790, and has been the largest U.S. city since 1790.

In [47]:
text_str = '''
Those Who Are Resilient Stay In The Game Longer
“On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” — Friedrich Nietzsche
Challenges and setbacks are not meant to defeat you, but promote you. However, I realise after many years of defeats, it can crush your spirit and it is easier to give up than risk further setbacks and disappointments. Have you experienced this before? To be honest, I don’t have the answers. I can’t tell you what the right course of action is; only you will know. However, it’s important not to be discouraged by failure when pursuing a goal or a dream, since failure itself means different things to different people. To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Same failure, yet different responses. Who is right and who is wrong? Neither. Each person has a different mindset that decides their outcome. Those who are resilient stay in the game longer and draw on their inner means to succeed.
I’ve coached mummy and mom clients who gave up after many years toiling away at their respective goal or dream. It was at that point their biggest breakthrough came. Perhaps all those years of perseverance finally paid off. It was the 19th Century’s minister Henry Ward Beecher who once said: “One’s best success comes after their greatest disappointments.” No one knows what the future holds, so your only guide is whether you can endure repeated defeats and disappointments and still pursue your dream. Consider the advice from the American academic and psychologist Angela Duckworth who writes in Grit: The Power of Passion and Perseverance: “Many of us, it seems, quit what we start far too early and far too often. Even more than the effort a gritty person puts in on a single day, what matters is that they wake up the next day, and the next, ready to get on that treadmill and keep going.”
I know one thing for certain: don’t settle for less than what you’re capable of, but strive for something bigger. Some of you reading this might identify with this message because it resonates with you on a deeper level. For others, at the end of their tether the message might be nothing more than a trivial pep talk. What I wish to convey irrespective of where you are in your journey is: NEVER settle for less. If you settle for less, you will receive less than you deserve and convince yourself you are justified to receive it.
“Two people on a precipice over Yosemite Valley” by Nathan Shipps on Unsplash
Develop A Powerful Vision Of What You Want
“Your problem is to bridge the gap which exists between where you are now and the goal you intend to reach.” — Earl Nightingale
I recall a passage my father often used growing up in 1990s: “Don’t tell me your problems unless you’ve spent weeks trying to solve them yourself.” That advice has echoed in my mind for decades and became my motivator. Don’t leave it to other people or outside circumstances to motivate you because you will be let down every time. It must come from within you. Gnaw away at your problems until you solve them or find a solution. Problems are not stop signs, they are advising you that more work is required to overcome them. Most times, problems help you gain a skill or develop the resources to succeed later. So embrace your challenges and develop the grit to push past them instead of retreat in resignation. Where are you settling in your life right now? Could you be you playing for bigger stakes than you are? Are you willing to play bigger even if it means repeated failures and setbacks? You should ask yourself these questions to decide whether you’re willing to put yourself on the line or settle for less. And that’s fine if you’re content to receive less, as long as you’re not regretful later.
If you have not achieved the success you deserve and are considering giving up, will you regret it in a few years or decades from now? Only you can answer that, but you should carve out time to discover your motivation for pursuing your goals. It’s a fact, if you don’t know what you want you’ll get what life hands you and it may not be in your best interest, affirms author Larry Weidel: “Winners know that if you don’t figure out what you want, you’ll get whatever life hands you.” The key is to develop a powerful vision of what you want and hold that image in your mind. Nurture it daily and give it life by taking purposeful action towards it.
Vision + desire + dedication + patience + daily action leads to astonishing success. Are you willing to commit to this way of life or jump ship at the first sign of failure? I’m amused when I read questions written by millennials on Quora who ask how they can become rich and famous or the next Elon Musk. Success is a fickle and long game with highs and lows. Similarly, there are no assurances even if you’re an overnight sensation, to sustain it for long, particularly if you don’t have the mental and emotional means to endure it. This means you must rely on the one true constant in your favour: your personal development. The more you grow, the more you gain in terms of financial resources, status, success — simple. If you leave it to outside conditions to dictate your circumstances, you are rolling the dice on your future.
So become intentional on what you want out of life. Commit to it. Nurture your dreams. Focus on your development and if you want to give up, know what’s involved before you take the plunge. Because I assure you, someone out there right now is working harder than you, reading more books, sleeping less and sacrificing all they have to realise their dreams and it may contest with yours. Don’t leave your dreams to chance.
'''

## Method

### 1.Create the word frequency table

In [40]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize


In [ ]:
def _create_frequency_table(text_string) -> dict:
    """
    we create a dictionary for the word frequency table.
    For this, we should only use the words that are not part of the stopWords array.
    Removing stop words and making frequency table
    Stemmer - an algorithm to bring words to its root word.
    :rtype: dict
    """
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    return freqTable


### 2.Tokenize the sentences

In [ ]:
#sent_tokenize(text_string)

### 3.Score the sentences: Term frequency

We’re using the Term Frequency method to score each sentence.
Basic Algorithm: score a sentence by its words, adding the frequency of every non-stop word in a sentence.

In [ ]:
def _score_sentences(sentences, freqTable) -> dict:
    """
    score a sentence by its words
    Basic algorithm: adding the frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = dict()

    for sentence in sentences:
        word_count_in_sentence = (len(word_tokenize(sentence)))
        word_count_in_sentence_except_stop_words = 0
        for wordValue in freqTable:
            if wordValue in sentence.lower():
                word_count_in_sentence_except_stop_words += 1
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    sentenceValue[sentence[:10]] = freqTable[wordValue]

        if sentence[:10] in sentenceValue:
            sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] / word_count_in_sentence_except_stop_words

        '''
        Notice that a potential issue with our score algorithm is that long sentences will have an advantage over short sentences. 
        To solve this, we're dividing every sentence score by the number of words in the sentence.
        
        Note that here sentence[:10] is the first 10 character of any sentence, this is to save memory while saving keys of
        the dictionary.
        '''

    return sentenceValue


### 4.Find the threshold
Here, we are considering the average score of the sentences as a threshold. You can use other methods to calculate the threshold.

In [ ]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original text
    average = (sumValues / len(sentenceValue))

    return average


### 5.Generate the summary
Basic Algorithm: Select a sentence for a summarization, If the sentence score is more than the average score.

In [ ]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary


### 6.Run Summarization

In [ ]:
def run_summarization(text):
    # 1 Create the word frequency table
    freq_table = _create_frequency_table(text)

    '''
    We already have a sentence tokenizer, so we just need 
    to run the sent_tokenize() method to create the array of sentences.
    '''

    # 2 Tokenize the sentences
    sentences = sent_tokenize(text)

    # 3 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(sentences, freq_table)

    # 4 Find the threshold
    threshold = _find_average_score(sentence_scores)

    # 5 Important Algorithm: Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.3 * threshold)

    return summary

## Text Summarize: Run

In [41]:
result = run_summarization(text_str)
print(result)

 To be honest, I don’t have the answers. Same failure, yet different responses. Neither. Don’t leave it to other people or outside circumstances to motivate you because you will be let down every time. And that’s fine if you’re content to receive less, as long as you’re not regretful later. Commit to it. Nurture your dreams. Don’t leave your dreams to chance.


In [48]:
result = run_summarization(document)
print(result)

 
New York City (NYC), often called simply New York, is the most populous city in the United States. New York City has been described as the cultural, financial, and media capital of the world, 
significantly influencing commerce, entertainment, research, technology, education, politics, 
tourism, art, fashion, and sports. Home to the headquarters of the United Nations, 
New York is an important center for international diplomacy. As many as 800 languages are spoken in New York,
  making it the most linguistically diverse city in the world. New York is home to more than 3.2 million residents born outside the United States, the largest foreign-born population of any city in the world as of 2016. If the New York metropolitan area were a sovereign state, it would have the eighth-largest economy in the world. New York is home to the highest number of billionaires of any city in the world. New York City traces its origins to a trading post founded by colonists from the 
Dutch Republic in 16

# Bert Extractive Summarizer
https://pypi.org/project/bert-extractive-summarizer/

## Setup

In [2]:
!pip install bert-extractive-summarizer

     |████████████████████████████████| 2.1MB 13.5MB/s 
     |████████████████████████████████| 901kB 50.8MB/s 
     |████████████████████████████████| 3.3MB 51.7MB/s 


In [3]:
!pip install spacy
!pip install transformers # > 2.2.0
!pip install neuralcoref

!python -m spacy download en_core_web_md

     |████████████████████████████████| 286kB 13.0MB/s 
     |████████████████████████████████| 133kB 28.7MB/s 
     |████████████████████████████████| 7.4MB 28.9MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
ERROR: botocore 1.20.54 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 96.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp37-none-any.whl size=98051305 sha256=f223e72428bb4f2f03f349222bf2bd8ae7d4f72b051ffa3e56a433cae54a9106
  Stored in directory: /tmp/pip-ephem-wheel-cache-v7sz5_ub/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [4]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 13.2MB/s 


## Text-Input

In [5]:
from summarizer import Summarizer

In [10]:
document = """
New York City (NYC), often called simply New York, is the most populous city in the United States. 
With an estimated 2019 population of 8,336,817 distributed over about 302.6 square miles (784 km2), 
New York City is also the most densely populated major city in the United States. 
Located at the southern tip of the U.S. state of New York, the city is the center of the 
New York metropolitan area, the largest metropolitan area in the world by urban landmass. 
With almost 20 million people in its metropolitan statistical area and approximately 
23 million in its combined statistical area, it is one of the world's most populous megacities. 
New York City has been described as the cultural, financial, and media capital of the world, 
significantly influencing commerce, entertainment, research, technology, education, politics, 
tourism, art, fashion, and sports. Home to the headquarters of the United Nations, 
New York is an important center for international diplomacy.
Situated on one of the world's largest natural harbors, 
New York City is composed of five boroughs, each of which is a county of the State of New York. 
The five boroughs—Brooklyn, Queens, Manhattan, the Bronx, and Staten Island—were consolidated 
into a single city in 1898. The city and its metropolitan area constitute the premier gateway for
 legal immigration to the United States. As many as 800 languages are spoken in New York,
  making it the most linguistically diverse city in the world. New York is home to more than 3.2 million residents born outside the United States, the largest foreign-born population of any city in the world as of 2016. As of 2019, the New York metropolitan area is estimated to produce a gross metropolitan product (GMP) of $2.0 trillion. If the New York metropolitan area were a sovereign state, it would have the eighth-largest economy in the world. New York is home to the highest number of billionaires of any city in the world.
New York City traces its origins to a trading post founded by colonists from the 
Dutch Republic in 1624 on Lower Manhattan; the post was named New Amsterdam in 1626.
 The city and its surroundings came under English control in 1664 and were renamed New York 
 after King Charles II of England granted the lands to his brother, the Duke of York. The city was regained
  by the Dutch in July 1673 and was subsequently renamed New Orange for one year and three months; 
  the city has been continuously named New York since November 1674. New York City was the capital of the United States from 1785 until 1790, and has been the largest U.S. city since 1790. The Statue of Liberty greeted millions of immigrants as they came to the U.S. by ship in the late 19th and early 20th centuries, and is a symbol of the U.S. and its ideals of liberty and peace. In the 21st century, New York has emerged as a global node of creativity, entrepreneurship, and environmental sustainability, and as a symbol of freedom and cultural diversity. In 2019, New York was voted the greatest city in the world per a survey of over 30,000 people from 48 cities worldwide, citing its cultural diversity.
Many districts and landmarks in New York City are well known, including three of the world's
 ten most visited tourist attractions in 2013. A record 62.8 million tourists visited New York City in 2017. 
 Times Square is the brightly illuminated hub of the Broadway Theater District,
  one of the world's busiest pedestrian intersections, and a major center of the world's entertainment industry.
   Many of the city's landmarks, skyscrapers, and parks are known around the world. 
   Manhattan's real estate market is among the most expensive in the world. 
   Providing continuous 24/7 service and contributing to the nickname The City that Never Sleeps, 
   the New York City Subway is the largest single-operator rapid transit system worldwide, 
   with 472 rail stations. The city has over 120 colleges and universities, 
   including Columbia University, New York University, Rockefeller University, 
   and the City University of New York system, which is the largest urban public 
   university system in the United States. Anchored by Wall Street in the Financial District of Lower Manhattan, New York City has been called both the world's leading financial center and the most financially 
powerful city in the world, and is home to the world's 
two largest stock exchanges by total market capitalization, the New York Stock Exchange and NASDAQ.
"""


New York City (NYC), often called simply New York, is the most populous city in the United States. The five boroughs—Brooklyn, Queens, Manhattan, the Bronx, and Staten Island—were consolidated 
into a single city in 1898. The city and its metropolitan area constitute the premier gateway for
 legal immigration to the United States. New York City traces its origins to a trading post founded by colonists from the 
Dutch Republic in 1624 on Lower Manhattan; the post was named New Amsterdam in 1626. In 2019, New York was voted the greatest city in the world per a survey of over 30,000 people from 48 cities worldwide, citing its cultural diversity. Many districts and landmarks in New York City are well known, including three of the world's
 ten most visited tourist attractions in 2013.

In [8]:

body = '''
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.
The deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.
Mubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.
Real estate firm Tishman Speyer had owned the other 10%.
The buyer is RFR Holding, a New York real estate company.
Officials with Tishman and RFR did not immediately respond to a request for comments.
It's unclear when the deal will close.
The building sold fairly quickly after being publicly placed on the market only two months ago.
The sale was handled by CBRE Group.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.
The rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.
Meantime, rents in the building itself are not rising nearly that fast.
While the building is an iconic landmark in the New York skyline, it is competing against newer office towers with large floor-to-ceiling windows and all the modern amenities.
Still the building is among the best known in the city, even to people who have never been to New York.
It is famous for its triangle-shaped, vaulted windows worked into the stylized crown, along with its distinctive eagle gargoyles near the top.
It has been featured prominently in many films, including Men in Black 3, Spider-Man, Armageddon, Two Weeks Notice and Independence Day.
The previous sale took place just before the 2008 financial meltdown led to a plunge in real estate prices.
Still there have been a number of high profile skyscrapers purchased for top dollar in recent years, including the Waldorf Astoria hotel, which Chinese firm Anbang Insurance purchased in 2016 for nearly $2 billion, and the Willis Tower in Chicago, which was formerly known as Sears Tower, once the world's tallest.
Blackstone Group (BX) bought it for $1.3 billion 2015.
The Chrysler Building was the headquarters of the American automaker until 1953, but it was named for and owned by Chrysler chief Walter Chrysler, not the company itself.
Walter Chrysler had set out to build the tallest building in the world, a competition at that time with another Manhattan skyscraper under construction at 40 Wall Street at the south end of Manhattan. He kept secret the plans for the spire that would grace the top of the building, building it inside the structure and out of view of the public until 40 Wall Street was complete.
Once the competitor could rise no higher, the spire of the Chrysler building was raised into view, giving it the title.
'''


"""
The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price. 
The building sold fairly quickly after being publicly placed on the market only two months ago.
The incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.'
Still the building is among the best known in the city, even to people who have never been to New York.
"""

In [7]:
body = '''
In a self-published 2015 autobiography, Mellon called social safety net programs “Slavery Redux,” adding: “For delivering their votes in the Federal Elections, they are awarded with yet more and more freebies: food stamps, cell phones, WIC payments, Obamacare, and on, and on, and on. The largess is funded by the hardworking folks, fewer and fewer in number, who are too honest or too proud to allow themselves to sink into this morass.”

Advertisement

Story continues below advertisement

Mellon declined to comment.

The Wyoming-based donor, whose family fortune dates to the Gilded Age, gave his first major pro-Trump donation in April, with a $10 million check to America First Action, the main super PAC supporting the president’s reelection. His donations are the biggest known contributions to the group by far, and he is also a top donor to GOP congressional super PACs, according to campaign finance records.

America First Action, a super PAC chaired by Linda McMahon, former head of the Small Business Administration, declined to comment on Mellon’s contribution.

Mellon’s company, Pan Am Systems, declined to comment on Mellon’s donation or support for Trump. The company did not respond to multiple requests for comment on the views expressed in his autobiography, which it described on its website as “a refreshingly candid look into his family life as well as his business successes.”

Advertisement

Story continues below advertisement

The book was available for free download on the company’s website until this week, when it was removed after inquiries by The Washington Post. Copies were still available through a separate website.

Mellon, who is the great-grandson of Mellon family patriarch and banker Thomas Mellon, and grandson of Andrew W. Mellon, the former Treasury Department secretary, had given smaller amounts to state and federal GOP candidates for years, but ramped up his giving under Trump, campaign finance records show. His first major federal donation came in May 2018, when he gave $10 million in support of the super PAC that supports the House GOP.

Since February 2018, he has given $40 million to three super PACs, and tens of thousands of dollars more to an array of GOP candidates, records show.

Story continues below advertisement

He now rivals other prominent donors who have increased their political giving under Trump, such as shipping supplies magnate Richard Uihlein and Stephen Schwarzman, the Blackstone chairman and chief executive.

Advertisement

In his autobiography, Mellon wrote that while his family had been Republicans for generations before him, it wasn’t until the presidency of Ronald Reagan that he fully considered himself a Republican. He said Reagan “understood that people did best for themselves when shackled with the least amount of governmental constraints.”

“Something had obviously gone dreadfully wrong with the Great Society and the Liberal onslaught. Poor people had become no less poor. Black people, in spite of heroic efforts by the ‘Establishment’ to right the wrongs of the past, became even more belligerent and unwilling to pitch in to improve their own situations,” Mellon wrote, describing his view of America during Reagan’s 1984 reelection campaign.

Story continues below advertisement

He continued: “Drugs rose to the level of epidemic. Single parent families became more and more prevalent. The likes of Jesse Jackson and Al Sharpton pandered endlessly to fan the flames.”

Advertisement

Two decades later, too many Americans are still relying on the government for help, he wrote.

Mellon slammed the educational system for becoming beholden to teachers unions, and wrote that “Black Studies, Women’s Studies, LGBT Studies, they have all cluttered Higher Education with a mishmash of meaningless tripe designed to brainwash gullible young adults into going along with the Dependency Syndrome.”

And he blasted media outlets, blaming journalists, particularly at MSNBC, for perpetuating the federal government’s “Dependency Message.”

Story continues below advertisement

“It took Abraham Lincoln and the Republican Party to deal with the first scourge of slavery. And now it appears that it is again up to the Republican Party to deal with the contemporary counterpart,” he wrote. “The question is: Is the Republican Party up to it this time?”

Advertisement

Mellon is the chairman of New Hampshire-based Pan Am Systems Inc., a privately held transportation and freight holding company whose subsidiaries include Pan Am Railways, a New England rail system, and a wood products manufacturer.

In the 1980s, Mellon took over a rail company called Guilford Transportation Industries and rebranded it after purchasing Pan Am Airways from bankruptcy in the 1990s.

Story continues below advertisement

Mellon’s company has publicly praised the GOP-backed 2017 tax overhaul bill, which represented the largest one-time reduction in the corporate tax rate in U.S. history and lowered taxes for the vast majority of Americans.

In May 2018 — the month he made his first donation to the Congressional Leadership Fund, the House GOP super PAC — his company announced it would issue a bonus of $1,100 to each employee because of the tax law.

Advertisement

He gave another $10 million to the Congressional Leadership Fund in November 2019. This year, he gave an additional $10 million to the Senate Leadership Fund, the Senate GOP super PAC.

Even as his political giving has increased, Mellon has maintained a low profile. When officials with the Congressional Leadership Fund invited Mellon to attend a donor retreat in Jackson Hole, Wyo., to express thanks for his 2018 donation and meet with then-House Speaker Paul D. Ryan, Mellon declined, according to someone familiar with the invitation who spoke on the condition of anonymity to describe private conversations.

Story continues below advertisement

“It was all over email, and he didn’t want to meet, and said no to meetings,” said the person familiar with Mellon’s donations, which came in response to an email to major GOP donors, the person said. “He’s very off-the-grid.”

Advertisement

Officials with the Senate Leadership Fund declined to comment. The Congressional Leadership Fund did not respond to requests for comment.

In an unusual move, Mellon has also given donations to two Democrats: $2,700 to Rep. Alexandria Ocasio-Cortez of New York for her 2018 campaign and $2,800 to Rep. Tulsi Gabbard of Hawaii for her long-shot bid for the 2020 Democratic presidential nomination.

Gabbard’s office did not ­respond to a request for comment. Ocasio-Cortez’s campaign spokeswoman, Lauren Hitt, said the campaign did not solicit the donation in 2018 and would return it after an inquiry by The Post.

Story continues below advertisement

In the 2016 GOP primaries, Mellon made donations in support of business executive Carly Fiorina and now-Secretary of Housing and Urban Development Ben Carson, before giving $2,700 in total to Trump and the Republican National Committee in the general election, records show.

Advertisement

In 2010, Mellon made headlines as the biggest donor to a legal fund set up by then-Arizona Gov. Jan Brewer (R) to defend Senate Bill 1070, a controversial immigration law that required police to determine the immigration status of people suspected of being in the country illegally, which critics had said would lead to rampant racial profiling. He gave an unsolicited $1.5 million to the fund, according to a person familiar with fundraising who spoke on the condition of anonymity to describe donor support.

“We were told by a representative that he was very closely watching current events and was inspired by [Brewer] and her leadership, and engaged on it,” the person said.

Alice Crites contributed to this report.
'''

## Model Load

In [8]:
model = Summarizer()

## Text Summarize: Run

In [9]:

result = model(body, min_length=60)
full = ''.join(result)
print(full)

In a self-published 2015 autobiography, Mellon called social safety net programs “Slavery Redux,” adding: “For delivering their votes in the Federal Elections, they are awarded with yet more and more freebies: food stamps, cell phones, WIC payments, Obamacare, and on, and on, and on. Advertisement

Story continues below advertisement

Mellon declined to comment. Mellon, who is the great-grandson of Mellon family patriarch and banker Thomas Mellon, and grandson of Andrew W. Mellon, the former Treasury Department secretary, had given smaller amounts to state and federal GOP candidates for years, but ramped up his giving under Trump, campaign finance records show. Story continues below advertisement

He now rivals other prominent donors who have increased their political giving under Trump, such as shipping supplies magnate Richard Uihlein and Stephen Schwarzman, the Blackstone chairman and chief executive. Black people, in spite of heroic efforts by the ‘Establishment’ to right the wrong